In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
df_train = pd.read_csv('preprocessed_train.csv', index_col='TX_ID')
df_test = pd.read_csv('preprocessed_test.csv')

# Keep a copy of TX_ID in X_test for saving predictions later
X_test_ids = df_test['TX_ID']

# Remove rows with missing target, separate target from predictors
df_train.dropna(axis=0, subset=['TX_FRAUD'], inplace=True)
y = df_train.TX_FRAUD
df_train.drop(['TX_FRAUD'], axis=1, inplace=True)

# Columns to drop that cannot be encoded or contain missing values
drop_cols = [
  'CUSTOMER_ID',
  'TERMINAL_ID',
  'CARD_DATA',
  'CARD_EXPIRY_DATE',
  'FAILURE_CODE',
  'FAILURE_REASON',
  'ACQUIRER_ID',
  'CARDHOLDER_AUTH_METHOD',
  'MERCHANT_ID'
]

# Drop the unwanted columns from both train and test sets
df_train.drop(drop_cols, axis=1, inplace=True)
df_test.drop(drop_cols, axis=1, inplace=True)

# Split the training data into train and validation sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, train_size=0.8, test_size=0.2, random_state=0)

# List of numeric and object columns
numeric_cols = [
  'TX_AMOUNT',
  'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
  'TRANSACTION_CASHBACK_AMOUNT',
]

object_cols = [ # Apply one-shot encoding
  'CARD_BRAND',
  'TRANSACTION_TYPE',
  'TRANSACTION_STATUS',
  'TRANSACTION_CURRENCY',
  'CARD_COUNTRY_CODE',
  'IS_RECURRING_TRANSACTION'
]

# Combine numeric and categorical columns into a single list for further processing
my_cols = numeric_cols + object_cols

# Select the relevant columns for train, validation, and test sets
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = df_test[my_cols].copy()

# One-hot encode categorical columns only (using pandas get_dummies)
X_train = pd.get_dummies(X_train, columns=object_cols)
X_valid = pd.get_dummies(X_valid, columns=object_cols)
X_test = pd.get_dummies(X_test, columns=object_cols)

# Align the columns of training, validation, and test sets
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Re-attach TX_ID to X_test for later use in predictions
X_test['TX_ID'] = X_test_ids

print(X_test.columns)
print()
print(X_valid.columns)


Index(['TX_AMOUNT', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
       'TRANSACTION_CASHBACK_AMOUNT', 'CARD_BRAND_AMEX', 'CARD_BRAND_Discover',
       'CARD_BRAND_MasterCard', 'CARD_BRAND_Visa',
       'TRANSACTION_TYPE_Cash Advance/Withdrawal', 'TRANSACTION_TYPE_Purchase',
       'TRANSACTION_TYPE_Purchase with cashback', 'TRANSACTION_TYPE_Refund',
       'TRANSACTION_STATUS_Authorized', 'TRANSACTION_STATUS_Captured',
       'TRANSACTION_STATUS_Rejected', 'TRANSACTION_STATUS_Settled',
       'TRANSACTION_CURRENCY_AED', 'TRANSACTION_CURRENCY_CAD',
       'TRANSACTION_CURRENCY_CHF', 'TRANSACTION_CURRENCY_CNY',
       'TRANSACTION_CURRENCY_EUR', 'TRANSACTION_CURRENCY_GBP',
       'TRANSACTION_CURRENCY_HKD', 'TRANSACTION_CURRENCY_JPY',
       'TRANSACTION_CURRENCY_MDL', 'TRANSACTION_CURRENCY_RON',
       'TRANSACTION_CURRENCY_USD', 'CARD_COUNTRY_CODE_AE',
       'CARD_COUNTRY_CODE_BE', 'CARD_COUNTRY_CODE_BT', 'CARD_COUNTRY_CODE_CA',
       'CARD_COUNTRY_CODE_DE', 'CARD_COUNTRY_CODE_DK', 'CAR

In [2]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor(random_state=0) # Your code here

# Fit the model
my_model_1.fit(X_train, y_train) # Your code here


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

In [3]:
from sklearn.metrics import mean_absolute_error

# Get predictions
predictions_1 = my_model_1.predict(X_valid) # Your code here


In [4]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 0.04875535606613984


In [5]:
# Define the model
my_model_2 = XGBRegressor(
  n_estimators=500,
  learning_rate=0.05,
  n_jobs=4,
  random_state=0,
  early_stopping_rounds=5,
  enable_categorical=True
) # Your code here

# Fit the model
my_model_2.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             verbose=False)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)  # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 0.04882967130248701


In [8]:
# Define the model
my_model_3 = XGBRegressor(
  n_estimators=500,
  learning_rate=0.005,
  n_jobs=10,
  random_state=0,
  early_stopping_rounds=5,
  enable_categorical=True
) # Your code here

# Fit the model
my_model_3.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)


Mean Absolute Error: 0.048835879731979934


In [7]:
# Drop the 'TX_ID' column from X_test before prediction
test_preds = my_model_3.predict(X_test.drop('TX_ID', axis=1))

# Create a submission DataFrame with 'TX_ID' and the predicted 'TX_FRAUD'
output = pd.DataFrame({
    'TX_ID': X_test.TX_ID,  # Retain TX_ID for the output
    'TX_FRAUD': test_preds
})

# Save to CSV, ensuring you don't set the index to 'TX_ID'
output.to_csv('submission.csv', index=False)
